In [17]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence, text
# fix random seed for reproducibility
numpy.random.seed(7)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [18]:
# read and replace null values
heavywater_dataset_df = pd.read_csv("/Users/channa/Projects/heavywater/classification_heavywater/data/shuffled-full-set-hashed.csv", 
            names = ['class', 'doc'])

heavywater_dataset_df.dropna(inplace = True)


In [19]:
# convert dataframe to matrix
# matrix values have tfidf of each word
num_words_to_keep = 2000
tokenizer_obj = text.Tokenizer(num_words = num_words_to_keep, 
                   filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', 
                   lower=True, 
                   split=' ', 
                   char_level=False, 
                   oov_token=None, 
                   document_count=0)
tokenizer_obj.fit_on_texts(heavywater_dataset_df.doc)
encoded_docs = tokenizer_obj.texts_to_matrix(heavywater_dataset_df.doc, mode='count')

In [20]:
heavywater_dataset_df.replace({'class' : { 'DELETION OF INTEREST' : 1, 
                                            'RETURNED CHECK' : 2, 
                                            'BILL' : 4, 
                                            'POLICY CHANGE' : 5, 
                                            'CANCELLATION NOTICE' : 6, 
                                            'DECLARATION' : 7, 
                                            'CHANGE ENDORSEMENT' : 8, 
                                            'NON-RENEWAL NOTICE' : 9, 
                                            'BINDER' : 10, 
                                            'REINSTATEMENT NOTICE' : 11, 
                                            'EXPIRATION NOTICE' : 12, 
                                            'INTENT TO CANCEL NOTICE' : 13, 
                                            'APPLICATION' : 14, 
                                            'BILL BINDER' : 0, }}, inplace=True)
y = np.array(heavywater_dataset_df['class'])

In [21]:
X_train, X_test, y_train, y_test = train_test_split(encoded_docs, y, test_size=0.33, random_state=42)

In [22]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((41646, 2000), (41646,), (20513, 2000), (20513,))

In [23]:
# # truncate and pad input sequences
# max_review_length = 2000
# X_train = sequence.pad_sequences(X_train, maxlen = max_review_length, dtype = 'int32')
# X_test = sequence.pad_sequences(X_test, maxlen = max_review_length, dtype = 'int32')

In [24]:
min(y_train)

0

In [27]:
# create the model
embedding_vector_length = 32
max_review_length = 2000
model = Sequential()
model.add(Embedding(num_words_to_keep, embedding_vector_length, input_length=max_review_length))
model.add(LSTM(32))
model.add(Dense(15, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=200, batch_size=1000)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 2000, 32)          64000     
_________________________________________________________________
lstm_4 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_4 (Dense)              (None, 15)                495       
Total params: 72,815
Trainable params: 72,815
Non-trainable params: 0
_________________________________________________________________
None
Train on 41646 samples, validate on 20513 samples
Epoch 1/200
41646/41646 [==============================] - 257s 6ms/step - loss: 2.2114 - acc: 0.3004 - val_loss: 2.0066 - val_acc: 0.3005
Epoch 2/200
41646/41646 [==============================] - 278s 7ms/step - loss: 1.9701 - acc: 0.3072 - val_loss: 1.9741 - val_acc: 0.3005
Epoch 3/200
41646/41646 [==============================] - 273s 7ms/s

Epoch 54/200
41646/41646 [==============================] - 261s 6ms/step - loss: 1.7824 - acc: 0.3366 - val_loss: 1.7797 - val_acc: 0.3372
Epoch 55/200
41646/41646 [==============================] - 239s 6ms/step - loss: 1.7793 - acc: 0.3350 - val_loss: 1.8258 - val_acc: 0.3156
Epoch 56/200
41646/41646 [==============================] - 242s 6ms/step - loss: 1.7740 - acc: 0.3349 - val_loss: 1.7711 - val_acc: 0.3333
Epoch 57/200
41646/41646 [==============================] - 236s 6ms/step - loss: 1.7691 - acc: 0.3371 - val_loss: 1.7800 - val_acc: 0.3301
Epoch 58/200
41646/41646 [==============================] - 247s 6ms/step - loss: 1.7669 - acc: 0.3381 - val_loss: 1.8908 - val_acc: 0.2603
Epoch 59/200
41646/41646 [==============================] - 272s 7ms/step - loss: 1.7629 - acc: 0.3392 - val_loss: 1.7973 - val_acc: 0.3236
Epoch 60/200
41646/41646 [==============================] - 273s 7ms/step - loss: 1.7554 - acc: 0.3461 - val_loss: 1.7974 - val_acc: 0.3298
Epoch 61/200
41646/4

41646/41646 [==============================] - 234s 6ms/step - loss: 1.7295 - acc: 0.3696 - val_loss: 1.7366 - val_acc: 0.3579
Epoch 113/200
41646/41646 [==============================] - 233s 6ms/step - loss: 1.7363 - acc: 0.3693 - val_loss: 1.7872 - val_acc: 0.3326
Epoch 114/200
41646/41646 [==============================] - 231s 6ms/step - loss: 1.7441 - acc: 0.3627 - val_loss: 1.7249 - val_acc: 0.3777
Epoch 115/200
41646/41646 [==============================] - 234s 6ms/step - loss: 1.7453 - acc: 0.3598 - val_loss: 1.7279 - val_acc: 0.3753
Epoch 116/200
41646/41646 [==============================] - 231s 6ms/step - loss: 1.7356 - acc: 0.3825 - val_loss: 1.7759 - val_acc: 0.3623
Epoch 117/200
41646/41646 [==============================] - 234s 6ms/step - loss: 1.7353 - acc: 0.3903 - val_loss: 2.3938 - val_acc: 0.3013
Epoch 118/200
41646/41646 [==============================] - 231s 6ms/step - loss: 1.8736 - acc: 0.3483 - val_loss: 1.8147 - val_acc: 0.3448
Epoch 119/200
41646/41646 [

41646/41646 [==============================] - 231s 6ms/step - loss: 1.6500 - acc: 0.4260 - val_loss: 1.7326 - val_acc: 0.4095
Epoch 171/200
41646/41646 [==============================] - 232s 6ms/step - loss: 1.6750 - acc: 0.4183 - val_loss: 1.7617 - val_acc: 0.4009
Epoch 172/200
41646/41646 [==============================] - 232s 6ms/step - loss: 1.6498 - acc: 0.4272 - val_loss: 1.6672 - val_acc: 0.4101
Epoch 173/200
41646/41646 [==============================] - 230s 6ms/step - loss: 1.6594 - acc: 0.4199 - val_loss: 1.6301 - val_acc: 0.4292
Epoch 174/200
41646/41646 [==============================] - 233s 6ms/step - loss: 1.6158 - acc: 0.4390 - val_loss: 1.6285 - val_acc: 0.4251
Epoch 175/200
41646/41646 [==============================] - 231s 6ms/step - loss: 1.6024 - acc: 0.4428 - val_loss: 1.6767 - val_acc: 0.4324
Epoch 176/200
41646/41646 [==============================] - 231s 6ms/step - loss: 1.5950 - acc: 0.4430 - val_loss: 1.6328 - val_acc: 0.4359
Epoch 177/200
41646/41646 [